### merging and combining all data

In [1]:
import os
import numpy as np
import pandas as pd
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Emory/NFL Lab/data/')

#### Full Merge

In [2]:
all_weeks = []

for root, dirs, files in os.walk('train/'):
    week_files = {}
    
    for file in files:
        if file.endswith('.csv'):
            week_num = file.split('_')[-1].replace('.csv', '')
            
            if week_num not in week_files:
                week_files[week_num] = {}
            
            file_path = os.path.join(root, file)
            
            if 'output' in file:
                week_files[week_num]['output'] = file_path
            else:
                week_files[week_num]['input'] = file_path

    # Process each week
    for week_num, paths in week_files.items():
        if 'input' in paths and 'output' in paths:
            df = pd.read_csv(paths['input'])
            df_throw = pd.read_csv(paths['output'])
            
            max_frames = df.groupby(['game_id','play_id', 'nfl_id'])['frame_id'].max().reset_index()
            max_frames.columns = ['game_id','play_id', 'nfl_id', 'max_frame_id']
            
            df_2_extended = df_throw.merge(max_frames, on=['game_id','play_id', 'nfl_id'], how='inner')
            df_2_extended['frame_id'] = df_2_extended['frame_id'] + df_2_extended['max_frame_id']
            
            df_combined = pd.concat([df, df_2_extended], ignore_index=True, sort=False)
            df_combined = df_combined.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id']).reset_index(drop=True).drop(columns=['max_frame_id'])
            df_combined.to_csv(f'combined/{week_num}_2023.csv', index=False)


In [3]:
comb = pd.DataFrame()
for root, dirs, files in os.walk('combined/'):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df_temp = pd.read_csv(file_path)
            comb = pd.concat([comb, df_temp], ignore_index=True)


In [4]:
comb['play_id_n'] = comb.groupby(['game_id','play_id']).ngroup()

In [5]:
missing_cols = ['player_height', 'player_weight', 'player_role', 'player_name', 'player_position','game_id', 'player_to_predict',
                'play_direction', 'absolute_yardline_number', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'player_side', 'play_id_n']

for col in missing_cols:
    if col in comb.columns:
        comb[col] = comb.groupby(['play_id_n', 'nfl_id'])[col].transform(lambda x: x.fillna(x.iloc[0] if x.notna().any() else np.nan))

C:\Users\dalto\AppData\Local\Temp\ipykernel_11128\1420306899.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  comb[col] = comb.groupby(['play_id_n', 'nfl_id'])[col].transform(lambda x: x.fillna(x.iloc[0] if x.notna().any() else np.nan))


In [6]:
df.loc[df['play_direction'] == 'left', 'x'] = 120 - df.loc[df['play_direction'] == 'left', 'x']
df.loc[df['play_direction'] == 'left', 'ball_land_x'] = 120 - df.loc[df['play_direction'] == 'left', 'ball_land_x']

In [7]:
comb.to_csv('2023_tracking.csv',index=False)